In [ ]:
import os, sys
import numpy as np
import xarray 
import rioxarray
import holoviews as hv
from holoviews import opts
import geoviews as gv
import datashader as ds
import cartopy.crs as ccrs
import hvplot
from holoviews.operation.datashader import regrid, shade
from bokeh.tile_providers import STAMEN_TONER
import rasterio
from osgeo import gdal
from pathlib import Path
import panel
import hvplot.xarray
from ipywidgets import interact, Dropdown, FloatSlider, IntSlider, SelectMultiple, Text
import matplotlib.pyplot as plt
import xmltodict
import zipfile
import tarfile
import pandas
import cv2
from skimage import exposure, img_as_float

# Import functions from common speclib functions file
sys.path.insert(0, os.path.abspath('../../'))
import speclib

hv.extension('bokeh', width=1000)

# DESIS L2

## Download from FTP
ftps://DESIS_Oliver@download.dsda.dlr.de//dims_op_oc_dfd2_694312879_1.tar.gz
ftps://DESIS_Oliver@download.dsda.dlr.de//dims_op_oc_dfd2_694398439_1.tar.gz

In [ ]:
usernameText = Text(value='username')
passwordText = Text(value='password')
filename = Text(value='filename')
display(usernameText)
display(passwordText)

In [ ]:
username = usernameText.value
password = passwordText.value

In [ ]:
!wget --user=$username --password=$password ftps://DESIS_Oliver@download.dsda.dlr.de//dims_op_oc_dfd2_694398439_1.tar.gz

## Unzip

In [ ]:
targzfiles = []
targzfiles += [each for each in os.listdir() if each.endswith('.tar.gz')]

In [ ]:
targzfiles

In [ ]:
# Extract the Zip archive from the tar.gz - extract the required files from the Zip archive
filenames = []
for filename in targzfiles:
    desistarfile = tarfile.open(filename, 'r')
    for i in desistarfile.getnames():
        if 'zip' in i:
            print(i)
            desistarfilezip = i
            if not Path(desiszipfile.filename).is_file():
                desiszipfile = zipfile.ZipFile(desistarfilezip, 'r')
            else:
                desistarfile.extract(desistarfilezip)
            
            if not Path(desiszipfile.filename).is_file():
                desistarfile.extract(desiszipfile.filename)
            
            for index, value in enumerate(desiszipfile.filelist):
                if 'SPECTRAL_IMAGE.tif' in str(value.filename):
                    desisspectralzipfile = value.filename
                if 'METADATA.xml' in str(value.filename):
                    desismetadatazipfile = str(value.filename)
            
            if Path(desisspectralzipfile).is_file():
                filename = desisspectralzipfile

            else:
                filename = desiszipfile.extract(desisspectralzipfile) 
            
            if Path(desismetadatazipfile).is_file():
                filenamexml = desiszipfile.extract(desismetadatazipfile)   

            
            file = filename.split('/')[-1]

            filenames.append({'file':file,'filename':filename,'xml': filenamexml})

In [ ]:
filenamespd = pandas.DataFrame(filenames)

In [ ]:
filenamesText = Dropdown(options = list(filenamespd.file))
display(filenamesText)

In [ ]:
filename = (filenamespd[filenamespd['file']==filenamesText.value]).reset_index().filename[0]
filenamexml = (filenamespd[filenamespd['file']==filenamesText.value]).reset_index().xml[0]

In [ ]:
fileptr = open(filenamexml,"r")
xml_content= fileptr.read()
desisdict = xmltodict.parse(xml_content)

In [ ]:
#desisdict

## Create useful band names and apply to GeoTIFF

In [ ]:
bands = []
for item in desisdict['hsi_doc']['specific']['bandCharacterisation']['band']:
    print(item['bandNumber'], item['wavelengthCenterOfBand'])#, item['response'], item['wavelengths'], item['gainOfBand'], item['offsetOfBand'])
    bandnumber = (int(item['bandNumber']))
    bands.append((bandnumber, 'VNIR'+str(bandnumber)+' '+str(int(item['wavelengthCenterOfBand'].split('.')[0]))))

In [ ]:
def set_band_descriptions(filepath, bands):
    """
    filepath: path/virtual path/uri to raster
    bands:    ((band, description), (band, description),...)
    """
    ds = gdal.Open(filepath, gdal.GA_Update)
    for band, desc in bands:
        rb = ds.GetRasterBand(band)
        rb.SetDescription(desc)
    del ds

In [ ]:
# Update the GeoTIFF band names to enable easier correspondence with other bands - use cwl for bands from metadata
speclib.set_band_descriptions(filename, bands)

In [ ]:
#gdal.Info(filename).split('\n')

In [ ]:
geofilename = filename.replace('.tif', '_geo.tif')
mercfilename = filename.replace('.tif', '_3857.tif')

## Warp GeoTIFF to lat/lon

In [ ]:
if not Path(geofilename).is_file():
    gdal.Warp(geofilename, filename, options="-t_srs EPSG:4326 -overwrite -tr 0.0003 0.0003 -wo WRITE_FLUSH=YES")
if not Path(mercfilename).is_file():
    gdal.Warp(mercfilename, filename, options="-t_srs EPSG:3857 -overwrite -wo WRITE_FLUSH=YES")

In [ ]:
desisxarray =  xarray.open_dataset(filename)

In [ ]:
desisxarraygeo = xarray.open_dataset(geofilename)

In [ ]:
desisxarraymerc =  xarray.open_dataset(mercfilename)

In [ ]:
desisxarraymerc

In [ ]:
speclib.rgbhsi(desisxarraymerc.band_data, 210, 85, 210, 0.1, 0.99).hvplot.rgb( x='x', y='y', bands='band', data_aspect=1, flip_yaxis=False, xaxis=False, yaxis=None, title="RGB Plot with HVPlot", width=1200)

In [ ]:
speclib.rgbhsi(desisxarraygeo.band_data, 150, 90, 50, 0.1, 0.99).hvplot.rgb( geo=True, x='x', y='y', bands='band',  title="RGB Plot with HVPlot", width=1200, tiles='ESRI', alpha=0.5)

## Make an interactive RGB plot

In [ ]:
# TODO
redW = Dropdown(options = list(desisxarray.band.values))#prismaxarray.band.values.tolist())
greenW = Dropdown(options = list(desisxarray.band.values))#prismaxarray.band.values.tolist())
blueW = Dropdown(options = list(desisxarray.band.values))#prismaxarray.band.values.tolist())
startW = Dropdown(options = ['Pause', 'Go'])

@interact(red = redW, green = greenW, blue = blueW, start = startW)

def rgb_combo(red, green, blue, start):
    redW.options = list(desisxarray.band.values)#prismaxarray.band.values.tolist()
    greenW.options = list(desisxarray.band.values)#prismaxarray.band.values.tolist()
    blueW.options = list(desisxarray.band.values)#prismaxarray.band.values.tolist()
    if start == 'Go' and red != green and green !=blue and red != blue:
        #(prismaxarray[[red,green,blue]]/100).to_array().hvplot.rgb(x='x', y='y', bands='variable', data_aspect=1, flip_yaxis=False, xaxis=False, yaxis=None)
        desisxarray.isel(band=[red,green,blue]).hvplot.rgb(x='x', y='y', bands='band', data_aspect=1, flip_yaxis=False, xaxis=False, yaxis=None, title="RGB Plot with HVPlot", width=1200)
    return(red,green,blue,start)